# Automated GenAI Newsletter Generator


## Setup Environment
Install dependencies:

In [ ]:
!pip install openai arxiv python-dotenv PyGithub python-pptx pandas --quiet

## Configuration
Enter your API keys:

In [ ]:
from google.colab import userdata
import os
import openai
from github import Github
import arxiv

# @title API Credentials
openai.api_key = userdata.get('OPENAI_API_KEY')
github_token = userdata.get('GITHUB_TOKEN')
email_password = userdata.get('EMAIL_PASSWORD')

# Initialize clients
github = Github(github_token)

## Core Functions
### 1. Content Aggregation
Fetch research papers and GitHub repos:

In [ ]:
def fetch_ai_content():
    """Fetch latest GenAI research and projects"""
    
    # Get arXiv papers
    arxiv_client = arxiv.Client()
    search = arxiv.Search(
        query="generative AI",
        max_results=5,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    papers = [result for result in arxiv_client.results(search)]

    # Get GitHub repos
    repos = github.search_repositories(
        query="generative AI created:>2024-02-20",
        sort="updated"
    )[:3]

    return {
        "papers": papers,
        "repos": repos
    }

### 2. AI-Powered Analysis
Generate technical summaries:

In [ ]:
def analyze_content(content):
    """Process content with GPT-4"""
    
    # Generate paper summaries
    paper_summaries = []
    for paper in content['papers']:
        response = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[{
                "role": "system",
                "content": f"Summarize this AI paper in 3 bullet points:\n{paper.summary}"
            }],
            temperature=0.3
        )
        paper_summaries.append(response.choices[0].message.content)

    # Analyze GitHub repos
    repo_analyses = []
    for repo in content['repos']:
        response = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[{
                "role": "system",
                "content": f"Explain the technical significance of {repo.name}:\n{repo.description}"
            }],
            temperature=0.5
        )
        repo_analyses.append(response.choices[0].message.content)

    return {
        "papers": paper_summaries,
        "repos": repo_analyses
    }

### 3. Newsletter Generation
Create formatted email content:

In [ ]:
def generate_newsletter(analysis):
    """Create HTML newsletter with AI insights"""
    
    # Generate header
    header = "# 🚀 Weekly GenAI Digest\n\n"
    
    # Top papers section
    papers_section = "## 📄 Top Research Papers\n"
    papers_section += "\n".join([f"- {summary}" for summary in analysis['papers']])
    
    # GitHub projects section
    repos_section = "\n\n## 💻 Trending Code Repos\n"
    repos_section += "\n".join([f"- {analysis}" for analysis in analysis['repos']])
    
    # Final assembly
    full_content = header + papers_section + repos_section
    
    # Format with GPT
    response = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"Convert this draft into professional newsletter format:\n{full_content}"
        }],
        temperature=0.7
    )
    
    return response.choices[0].message.content

## Execution Pipeline
Run the complete workflow:

In [ ]:
# @title Generate This Week's Issue
content = fetch_ai_content()
analysis = analyze_content(content)
newsletter = generate_newsletter(analysis)

print("Newsletter Generated!\n")
print(newsletter)

## Email Distribution
Send newsletter to subscribers:

In [ ]:
import smtplib
from email.mime.text import MIMEText

def send_newsletter(recipients):
    """Distribute via Gmail"""
    
    msg = MIMEText(newsletter, 'plain')
    msg['Subject'] = 'Your Weekly GenAI Update'
    msg['From'] = 'genai-news@yourdomain.com'
    msg['To'] = ', '.join(recipients)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login('your.email@gmail.com', email_password)
        server.sendmail(
            msg['From'], recipients, msg.as_string()
        )

# @title Send Test Email
test_recipients = ["your.email@gmail.com"]  # Replace with test email
send_newsletter(test_recipients)
print("Newsletter sent successfully!")